In [1]:
import json
import pickle
import re
import string
import urllib.request as ur
import warnings
from collections import Counter
from copy import deepcopy
from itertools import product
from operator import itemgetter
from pathlib import Path
import os
from glob import glob
from collections import Counter

warnings.filterwarnings("ignore")

import cv2
import face_recognition
import gensim
import gensim.downloader as api
import googleapiclient.discovery
import joblib
import keras_ocr
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import seaborn as sns
import skimage
from tqdm.notebook import tqdm
from gensim.models import Word2Vec
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from scipy.stats import iqr
from skimage import color, exposure, transform
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model
from tinydb import Query, TinyDB
from yolov4.tf import YOLOv4

from utils import YTDownloader, read_config

def load_pickle(path):
    print("Loading from:", path)
    with open(path, 'rb') as fh:
        return pickle.load(fh)

def save_pickle(obj, path):
    print("Saving to:", path)
    with open(path, 'wb') as fh:
        return pickle.dump(obj, fh)

cache_dir = Path('data/chkp5/cache')
cache_dir.mkdir(exist_ok=True, parents=True)

emotions_to_idx = {'Angry': 0, 'Sad': 5, 'Neutral': 4, 'Disgust': 1, 'Surprise': 6, 'Fear': 2, 'Happy': 3}
idx_to_emotions = {v:k for k,v in emotions_to_idx.items()}

config = read_config("config.json")
config['dataset_path'] = Path(config['dataset_path'])

start_transaction = lambda: TinyDB('data/not_trending_db.json')

In [2]:
wv = api.load('word2vec-google-news-300')
model = load_model('data/model_v6_23.hdf5')
pipeline = keras_ocr.pipeline.Pipeline()

Looking for /home/hylomorph/.keras-ocr/craft_mlt_25k.h5
Looking for /home/hylomorph/.keras-ocr/crnn_kurapan.h5


## Wybieranie unikalnych ID filmów trending

In [3]:
# df = pd.read_pickle('data/chkp1.pkl')

# with start_transaction() as db:
#     video_ids = set(map(itemgetter('video_id'), db.all()))
#     print("Liczba rekordów:", len(video_ids))

#     inserted = db.insert_multiple({'video_id': idx} for idx in df.index if idx not in video_ids)
#     print("Wstawiono:", len(inserted))

In [4]:
# minDate = df.publish_time.min().isoformat()
# maxDate = df.publish_time.max().isoformat()
# print("Wyszukiwanie danych od:", minDate, "do", maxDate)

## Wczytywanie danych

In [5]:
df = pd.read_json('data/chkp5/data.json') #.head(42)
df.set_index('video_id', inplace=True)
print(df.shape)
df.head(3)

(45208, 17)


,publishedAt,channelId,title,description,thumbnails,channelTitle,tags,categoryId,liveBroadcastContent,localized,defaultAudioLanguage,viewCount,likeCount,dislikeCount,favoriteCount,commentCount,defaultLanguage
video_id,,,,,,,,,,,,,,,,,
Y8Ceu9YBR1I,2019-01-02T14:00:03Z,UCIiBf-JbtCazHSFqXV4JgoA,GRACE N MICHELLE REUNITE AFTER 6 YEARS!!!! // ...,Michelle and I reunited AT LAST! And we did a ...,{'default': {'url': 'https://i.ytimg.com/vi/Y8...,Grace Helbig,"[grace, helbig, grace helbig, gracehelbig, gra...",22,none,{'title': 'GRACE N MICHELLE REUNITE AFTER 6 YE...,en,134287.0,9284.0,27.0,0,910.0,NaN
St6aYO0Gz5U,2019-02-09T05:35:24Z,UCIRYBXDze5krPDzAEOxFGVA,Matthew Whitaker testifies before House judici...,US acting attorney general Matthew Whitaker te...,{'default': {'url': 'https://i.ytimg.com/vi/St...,Guardian News,"[matthew whitaker, whitaker, whitaker testimon...",25,none,{'title': 'Matthew Whitaker testifies before H...,en-GB,31382.0,149.0,30.0,0,18.0,NaN
k8JuFit-j38,2018-09-06T16:00:13Z,UC1A_Hq-N1dHhAvwg0QWC7Sw,Making Music But Everything Is A Challenge,"I take a plugin which generates ""random beat m...",{'default': {'url': 'https://i.ytimg.com/vi/k8...,Levi Niha,"[Beat Making, Making A Beat, Music, Music Maki...",10,none,{'title': 'Making Music But Everything Is A Ch...,NaN,230180.0,6358.0,122.0,0,469.0,NaN


#### Ujednolicenie nazw kolumn

In [6]:
df = df[[
    'publishedAt', 'title', 'description',
    'channelTitle', 'categoryId', 'tags',
    'viewCount', 'likeCount', 'dislikeCount',
    'commentCount', 'thumbnails'
]]

df.columns = [
    'publish_time', 'title', 'description',
    'channel_title', 'category_id', 'tags',
    'views', 'likes', 'dislikes',
    'comment_count', 'thumbnails'
]

df.head(3)

,publish_time,title,description,channel_title,category_id,tags,views,likes,dislikes,comment_count,thumbnails
video_id,,,,,,,,,,,
Y8Ceu9YBR1I,2019-01-02T14:00:03Z,GRACE N MICHELLE REUNITE AFTER 6 YEARS!!!! // ...,Michelle and I reunited AT LAST! And we did a ...,Grace Helbig,22,"[grace, helbig, grace helbig, gracehelbig, gra...",134287.0,9284.0,27.0,910.0,{'default': {'url': 'https://i.ytimg.com/vi/Y8...
St6aYO0Gz5U,2019-02-09T05:35:24Z,Matthew Whitaker testifies before House judici...,US acting attorney general Matthew Whitaker te...,Guardian News,25,"[matthew whitaker, whitaker, whitaker testimon...",31382.0,149.0,30.0,18.0,{'default': {'url': 'https://i.ytimg.com/vi/St...
k8JuFit-j38,2018-09-06T16:00:13Z,Making Music But Everything Is A Challenge,"I take a plugin which generates ""random beat m...",Levi Niha,10,"[Beat Making, Making A Beat, Music, Music Maki...",230180.0,6358.0,122.0,469.0,{'default': {'url': 'https://i.ytimg.com/vi/k8...


#### Opracowanie miniaturek

In [7]:
def get_tbnl_url(x):
    return x['high']['url']

df['thumbnail_link'] = df['thumbnails'].apply(get_tbnl_url)
df.drop('thumbnails', axis=1, inplace=True)
df.head(3)

,publish_time,title,description,channel_title,category_id,tags,views,likes,dislikes,comment_count,thumbnail_link
video_id,,,,,,,,,,,
Y8Ceu9YBR1I,2019-01-02T14:00:03Z,GRACE N MICHELLE REUNITE AFTER 6 YEARS!!!! // ...,Michelle and I reunited AT LAST! And we did a ...,Grace Helbig,22,"[grace, helbig, grace helbig, gracehelbig, gra...",134287.0,9284.0,27.0,910.0,https://i.ytimg.com/vi/Y8Ceu9YBR1I/hqdefault.jpg
St6aYO0Gz5U,2019-02-09T05:35:24Z,Matthew Whitaker testifies before House judici...,US acting attorney general Matthew Whitaker te...,Guardian News,25,"[matthew whitaker, whitaker, whitaker testimon...",31382.0,149.0,30.0,18.0,https://i.ytimg.com/vi/St6aYO0Gz5U/hqdefault.jpg
k8JuFit-j38,2018-09-06T16:00:13Z,Making Music But Everything Is A Challenge,"I take a plugin which generates ""random beat m...",Levi Niha,10,"[Beat Making, Making A Beat, Music, Music Maki...",230180.0,6358.0,122.0,469.0,https://i.ytimg.com/vi/k8JuFit-j38/hqdefault.jpg


In [8]:
# df_marcin = pd.read_csv('data/df_marcin.csv').set_index('video_id')
# print(df_marcin.shape)

# df_marcin = df_marcin.loc[list(set(df_marcin[df.columns].index) - set(df.index))]
# print(df_marcin.shape)

In [9]:
# df = pd.concat([df, df_marcin], axis=0)
# print(df.shape)

In [10]:
def handle_dates(df, cols):
    for col in cols:
        df[col+'_day'] = df[col].dt.day
        df[col+'_month'] = df[col].dt.month
        df[col+'_year'] = df[col].dt.year
        df[col+'_weekday'] = df[col].dt.dayofweek
        df[col+'_quarter'] = df[col].dt.quarter
        if col == 'publish_time':
            df[col+'_hour'] = df[col].dt.hour
            df[col+'_minute'] = df[col].dt.minute
            df[col+'_second'] = df[col].dt.second
    return df

df['publish_time'] = pd.to_datetime(df['publish_time'])
df = handle_dates(df, ['publish_time'])
df.drop('publish_time', axis=1, inplace=True)
df.head(3)

,title,description,channel_title,category_id,tags,views,likes,dislikes,comment_count,thumbnail_link,publish_time_day,publish_time_month,publish_time_year,publish_time_weekday,publish_time_quarter,publish_time_hour,publish_time_minute,publish_time_second
video_id,,,,,,,,,,,,,,,,,,
Y8Ceu9YBR1I,GRACE N MICHELLE REUNITE AFTER 6 YEARS!!!! // ...,Michelle and I reunited AT LAST! And we did a ...,Grace Helbig,22,"[grace, helbig, grace helbig, gracehelbig, gra...",134287.0,9284.0,27.0,910.0,https://i.ytimg.com/vi/Y8Ceu9YBR1I/hqdefault.jpg,2,1,2019,2,1,14,0,3
St6aYO0Gz5U,Matthew Whitaker testifies before House judici...,US acting attorney general Matthew Whitaker te...,Guardian News,25,"[matthew whitaker, whitaker, whitaker testimon...",31382.0,149.0,30.0,18.0,https://i.ytimg.com/vi/St6aYO0Gz5U/hqdefault.jpg,9,2,2019,5,1,5,35,24
k8JuFit-j38,Making Music But Everything Is A Challenge,"I take a plugin which generates ""random beat m...",Levi Niha,10,"[Beat Making, Making A Beat, Music, Music Maki...",230180.0,6358.0,122.0,469.0,https://i.ytimg.com/vi/k8JuFit-j38/hqdefault.jpg,6,9,2018,3,3,16,0,13


### Wczytywanie danych trending

In [11]:
df_trending = pd.read_pickle('data/chkp2.pkl')
df_filled = pd.read_pickle('data/chkp12_filled.pkl')

df_trending.drop('category_id', axis=1, inplace=True)
df_trending = df_trending.join(df_filled.category_id)

df_trending['tags'] = df_trending.tags.str.split('|').apply(lambda x: [s.strip('"') for s in x])
df_trending = df_trending[df.columns]
df_trending.head(3)

,title,description,channel_title,category_id,tags,views,likes,dislikes,comment_count,thumbnail_link,publish_time_day,publish_time_month,publish_time_year,publish_time_weekday,publish_time_quarter,publish_time_hour,publish_time_minute,publish_time_second
video_id,,,,,,,,,,,,,,,,,,
9wRQljFNDW8,Dion Lewis' 103-Yd Kick Return TD vs. Denver! ...,New England Patriots returner Dion Lewis blast...,NFL,17.0,"[NFL, Football, offense, defense, afc, nfc, Am...",81377,655,25,177,https://i.ytimg.com/vi/9wRQljFNDW8/default.jpg,13,11,2017,0,4,2,5,26
Om_zGhJLZ5U,TL;DW - Every DCEU Movie Before Justice League,With Justice League approaching fast we rewatc...,Screen Junkies,1.0,"[screenjunkies, screen junkies, sj news, hones...",288922,7515,792,2111,https://i.ytimg.com/vi/Om_zGhJLZ5U/default.jpg,12,11,2017,6,4,18,0,3
goP4Z5wyOlM,Iraq-Iran earthquake: Deadly tremor hits borde...,A strong 7.2-magnitude earthquake has rattled ...,BBC News,25.0,"[bbc, bbc news, news, iran, iran news, iraq, i...",34785,308,26,413,https://i.ytimg.com/vi/goP4Z5wyOlM/default.jpg,12,11,2017,6,4,21,16,40


In [12]:
df['trending'] = 0
df_trending['trending'] = 1

df = pd.concat([df, df_trending], axis=0)
df['category_id'] = df.category_id.astype('int')
df.tags.fillna('[none]', inplace=True)
df.head(3)

,title,description,channel_title,category_id,tags,views,likes,dislikes,comment_count,thumbnail_link,publish_time_day,publish_time_month,publish_time_year,publish_time_weekday,publish_time_quarter,publish_time_hour,publish_time_minute,publish_time_second,trending
video_id,,,,,,,,,,,,,,,,,,,
Y8Ceu9YBR1I,GRACE N MICHELLE REUNITE AFTER 6 YEARS!!!! // ...,Michelle and I reunited AT LAST! And we did a ...,Grace Helbig,22,"[grace, helbig, grace helbig, gracehelbig, gra...",134287.0,9284.0,27.0,910.0,https://i.ytimg.com/vi/Y8Ceu9YBR1I/hqdefault.jpg,2,1,2019,2,1,14,0,3,0
St6aYO0Gz5U,Matthew Whitaker testifies before House judici...,US acting attorney general Matthew Whitaker te...,Guardian News,25,"[matthew whitaker, whitaker, whitaker testimon...",31382.0,149.0,30.0,18.0,https://i.ytimg.com/vi/St6aYO0Gz5U/hqdefault.jpg,9,2,2019,5,1,5,35,24,0
k8JuFit-j38,Making Music But Everything Is A Challenge,"I take a plugin which generates ""random beat m...",Levi Niha,10,"[Beat Making, Making A Beat, Music, Music Maki...",230180.0,6358.0,122.0,469.0,https://i.ytimg.com/vi/k8JuFit-j38/hqdefault.jpg,6,9,2018,3,3,16,0,13,0


# Wyznaczanie cech

In [13]:
text_features = ['title', 'description', 'channel_title']
attention_symbols = '!?$#'

def punctutation_count(text):
    return len([c for c in str(text) if c in string.punctuation])

def attention_count(text):
    return len([c for c in str(text) if c in attention_symbols])

def text_len(text):
    return len(str(text))

def letter_count(text):
    return len(list(filter(str.isalpha, str(text))))

def information_ratio(df):
    for name in text_features:
        df[name+'_information_ratio'] = (df[name+'_letter_count'] / df[name+'_len']).fillna(0)
    return df

def attention_ratio(df):
    for name in text_features:
        df[name+'_attention_ratio'] = (df[name+'_attention_count'] / df[name+'_letter_count']).fillna(0)
    return df

def word_count(df):
    for name in text_features:
        df[name+'_word_count'] = df[name].str.count(r"[\w\-_#\/\\\+\:$?]+")
    return df

def url_count(df):
    df['description_url_count'] = df.description.str.count(r"(https?:\/\/)?(\w+\.\w+)[\?=\&\w_\-.\/.]*")
    return df

def url_ratio(df):
    df['description_url_ratio'] = (df['description_url_count'] / df['description_word_count']).fillna(0)
    return df

def shouting_ratio(text):
    """Returns ratio of upper letters to all letters. Ignores non-letters in summary"""
    if not isinstance(text, str):
        return np.nan
    
    letters = ''.join(filter(str.isalpha, text))
    if letters:
        uppers = ''.join(filter(str.isupper, letters))
        return len(uppers) / len(letters)
    else:
        return 0

df[[n+'_punctuation_count' for n in text_features]] = df[text_features].applymap(punctutation_count)
df[[n+'_attention_count' for n in text_features]] = df[text_features].applymap(attention_count)
df[[n+'_len' for n in text_features]] = df[text_features].applymap(text_len)
df[[n+'_letter_count' for n in text_features]] = df[text_features].applymap(letter_count)
df = information_ratio(df)
df = attention_ratio(df)
df = word_count(df)
df = url_count(df)
df = url_ratio(df)
df[[f'shouting_ratio_{f}' for f in text_features]] = df[text_features].applymap(shouting_ratio)

df.head(3)

,title,description,channel_title,category_id,tags,views,likes,dislikes,comment_count,thumbnail_link,...,description_attention_ratio,channel_title_attention_ratio,title_word_count,description_word_count,channel_title_word_count,description_url_count,description_url_ratio,shouting_ratio_title,shouting_ratio_description,shouting_ratio_channel_title
video_id,,,,,,,,,,,,,,,,,,,,,
Y8Ceu9YBR1I,GRACE N MICHELLE REUNITE AFTER 6 YEARS!!!! // ...,Michelle and I reunited AT LAST! And we did a ...,Grace Helbig,22,"[grace, helbig, grace helbig, gracehelbig, gra...",134287.0,9284.0,27.0,910.0,https://i.ytimg.com/vi/Y8Ceu9YBR1I/hqdefault.jpg,...,0.007291,0.0,10,253.0,2,25.0,0.098814,0.785714,0.143578,0.181818
St6aYO0Gz5U,Matthew Whitaker testifies before House judici...,US acting attorney general Matthew Whitaker te...,Guardian News,25,"[matthew whitaker, whitaker, whitaker testimon...",31382.0,149.0,30.0,18.0,https://i.ytimg.com/vi/St6aYO0Gz5U/hqdefault.jpg,...,0.000000,0.0,9,82.0,2,8.0,0.097561,0.048387,0.064125,0.166667
k8JuFit-j38,Making Music But Everything Is A Challenge,"I take a plugin which generates ""random beat m...",Levi Niha,10,"[Beat Making, Making A Beat, Music, Music Maki...",230180.0,6358.0,122.0,469.0,https://i.ytimg.com/vi/k8JuFit-j38/hqdefault.jpg,...,0.004878,0.0,7,60.0,2,7.0,0.116667,0.194444,0.178049,0.250000


In [14]:
def _hash(object, trim):
    obj = hash(object)
    return np.float32(str(obj)[:trim])

for col, trim in zip(['channel_title'], [12]):
    df[col+'_hash'] = df[col].apply(_hash, trim=trim).astype(np.float32)

df.head(3)

,title,description,channel_title,category_id,tags,views,likes,dislikes,comment_count,thumbnail_link,...,channel_title_attention_ratio,title_word_count,description_word_count,channel_title_word_count,description_url_count,description_url_ratio,shouting_ratio_title,shouting_ratio_description,shouting_ratio_channel_title,channel_title_hash
video_id,,,,,,,,,,,,,,,,,,,,,
Y8Ceu9YBR1I,GRACE N MICHELLE REUNITE AFTER 6 YEARS!!!! // ...,Michelle and I reunited AT LAST! And we did a ...,Grace Helbig,22,"[grace, helbig, grace helbig, gracehelbig, gra...",134287.0,9284.0,27.0,910.0,https://i.ytimg.com/vi/Y8Ceu9YBR1I/hqdefault.jpg,...,0.0,10,253.0,2,25.0,0.098814,0.785714,0.143578,0.181818,8.459502e+11
St6aYO0Gz5U,Matthew Whitaker testifies before House judici...,US acting attorney general Matthew Whitaker te...,Guardian News,25,"[matthew whitaker, whitaker, whitaker testimon...",31382.0,149.0,30.0,18.0,https://i.ytimg.com/vi/St6aYO0Gz5U/hqdefault.jpg,...,0.0,9,82.0,2,8.0,0.097561,0.048387,0.064125,0.166667,2.520666e+11
k8JuFit-j38,Making Music But Everything Is A Challenge,"I take a plugin which generates ""random beat m...",Levi Niha,10,"[Beat Making, Making A Beat, Music, Music Maki...",230180.0,6358.0,122.0,469.0,https://i.ytimg.com/vi/k8JuFit-j38/hqdefault.jpg,...,0.0,7,60.0,2,7.0,0.116667,0.194444,0.178049,0.250000,-5.254257e+10


## Pobieranie miniaturek

In [15]:
count_unknown = 0

for index, row in tqdm(df.iterrows(), total=len(df.index)):
    path = config['dataset_path'] / 'images/{}.jpg'.format(index)
    link = row['thumbnail_link']
    try:
        if not os.path.exists(path):
            ur.urlretrieve(link, path)
    except:
        count_unknown += 1

print("Nieudane pobrania", count_unknown)

  0%|          | 0/53814 [00:00<?, ?it/s]

Nieudane pobrania 825


## Zaawansowane cechy

In [16]:
def mod_img(df, func, cache=True, as_df=True, colname='', *args):
    if isinstance(colname, list):
        fname = '_'.join(colname)+'.pkl'
    else:
        fname = colname+'.pkl'

    if (cache_dir / fname).exists() and cache:
        col_val_list = load_pickle(cache_dir / fname)
    else:
        col_val_list = []
        for index, row in tqdm(df.iterrows(), total=df.shape[0]):
            try:
                col_val_list.append(func(index, *args))
            except Exception as ex:
                print(ex)
                col_val_list.append(tuple(np.NaN for _ in colname) if isinstance(colname, list) else np.NaN)
        if cache:
            save_pickle(col_val_list, cache_dir / fname)

    if len(col_val_list) > 0 and isinstance(col_val_list[0], dict) and as_df:
        new_cols = pd.DataFrame(col_val_list, index=df.index)
        df = pd.concat([df, new_cols], axis=1)
    else:
        df[colname] = col_val_list
    return df

#### YOLO v4

In [18]:
yolo = YOLOv4()
yolo.config.parse_names("data/yolo_data/coco.names")
yolo.config.parse_cfg("data/yolo_data/yolov4-tiny.cfg")
yolo.make_model()
yolo.load_weights("data/yolo_data/yolov4-tiny.weights", weights_type="yolo")

def apply_yolo(index, prob_thresh=0.25):
    try:
        frame = cv2.imread("data/images/{}.jpg".format(index))    
        # print(frame)
        # print(frame.shape)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pred = yolo.predict(frame_rgb, prob_thresh=prob_thresh)
        objects = []
        max_prob_object = (np.nan, -1)
        for i in pred:
            cx, cy, hw, hh, label_id, prob = i
            if prob > prob_thresh:
                object_ = yolo.config.names[label_id]
                objects.append(object_)
                if prob > max_prob_object[1]: max_prob_object = (object_, prob)
        c = Counter(objects)
        most_common = c.most_common(1)[0][0]
    except IndexError:
        most_common = np.nan
    except:
        return 0, [], (np.nan, -1), np.nan
    finally:
        return c['person'], objects, max_prob_object, most_common

df = mod_img(df, apply_yolo, colname=['people_amount', 'objects', 'max_prob_object', 'most_occurence_object'])
df.head(5)

  0%|          | 0/53814 [00:00<?, ?it/s]

local variable 'c' referenced before assignment
local variable 'c' referenced before assignment
local variable 'c' referenced before assignment
local variable 'c' referenced before assignment
local variable 'c' referenced before assignment
local variable 'c' referenced before assignment
local variable 'c' referenced before assignment
local variable 'c' referenced before assignment
local variable 'c' referenced before assignment
local variable 'c' referenced before assignment
local variable 'c' referenced before assignment
local variable 'c' referenced before assignment
local variable 'c' referenced before assignment
local variable 'c' referenced before assignment
local variable 'c' referenced before assignment
local variable 'c' referenced before assignment
local variable 'c' referenced before assignment
local variable 'c' referenced before assignment
local variable 'c' referenced before assignment
local variable 'c' referenced before assignment
local variable 'c' referenced before ass

,title,description,channel_title,category_id,tags,views,likes,dislikes,comment_count,thumbnail_link,...,description_url_count,description_url_ratio,shouting_ratio_title,shouting_ratio_description,shouting_ratio_channel_title,channel_title_hash,people_amount,objects,max_prob_object,most_occurence_object
video_id,,,,,,,,,,,,,,,,,,,,,
Y8Ceu9YBR1I,GRACE N MICHELLE REUNITE AFTER 6 YEARS!!!! // ...,Michelle and I reunited AT LAST! And we did a ...,Grace Helbig,22,"[grace, helbig, grace helbig, gracehelbig, gra...",134287.0,9284.0,27.0,910.0,https://i.ytimg.com/vi/Y8Ceu9YBR1I/hqdefault.jpg,...,25.0,0.098814,0.785714,0.143578,0.181818,8.459502e+11,4.0,"[person, person, person, person]","(person, 0.9443111)",person
St6aYO0Gz5U,Matthew Whitaker testifies before House judici...,US acting attorney general Matthew Whitaker te...,Guardian News,25,"[matthew whitaker, whitaker, whitaker testimon...",31382.0,149.0,30.0,18.0,https://i.ytimg.com/vi/St6aYO0Gz5U/hqdefault.jpg,...,8.0,0.097561,0.048387,0.064125,0.166667,2.520666e+11,8.0,"[tie, person, person, person, person, person, ...","(person, 0.9942337)",person
k8JuFit-j38,Making Music But Everything Is A Challenge,"I take a plugin which generates ""random beat m...",Levi Niha,10,"[Beat Making, Making A Beat, Music, Music Maki...",230180.0,6358.0,122.0,469.0,https://i.ytimg.com/vi/k8JuFit-j38/hqdefault.jpg,...,7.0,0.116667,0.194444,0.178049,0.250000,-5.254257e+10,0.0,[],"(nan, -1)",NaN
_CzomU7kgFE,Doctor Fate Character Details & Powers Reveale...,Welcome to episode #3 of the DC Universe Annot...,HN Entertainment,24,"[Hybrid Network, HN Entertainment, HNE, Doctor...",1925.0,83.0,4.0,12.0,https://i.ytimg.com/vi/_CzomU7kgFE/hqdefault.jpg,...,3.0,0.018182,0.307692,0.088915,0.200000,1.813211e+11,1.0,"[dog, person]","(person, 0.60851365)",dog
s1bKD0kqPzk,Stephen Fry & Bill Bailey Hilariously Read The...,A little throwback to Stephen Fry & Bill Baile...,The Graham Norton Show,24,"[Graham Norton, Graham Norton Show, The Graham...",756989.0,8855.0,164.0,344.0,https://i.ytimg.com/vi/s1bKD0kqPzk/hqdefault.jpg,...,3.0,0.049180,0.179487,0.103627,0.210526,7.288290e+11,2.0,"[person, person]","(person, 0.7665016)",person


#### Kodowanie obiektów

In [19]:
to_encode = ['most_occurence_object']

moo_path = cache_dir / 'le_most_occurence_object.pkl'
if moo_path.exists():
    le_moo = load_pickle(moo_path)
    df[to_encode] = le_moo.transform(df[to_encode].values.ravel())    
else:
    le_moo = LabelEncoder()
    df[to_encode] = le_moo.fit_transform(df[to_encode].values.ravel())
    save_pickle(le_moo, moo_path)

Saving to: data/chkp5/cache/le_most_occurence_object.pkl


In [20]:
to_untuple = ['max_prob_object']

mpo_path = cache_dir / 'le_max_prob_object.pkl'
if mpo_path.exists():
    lee_mpo = load_pickle(mpo_path)
    df[to_untuple] = lee_mpo.transform(df[to_untuple].\
        applymap(lambda x: x[0] if isinstance(x, tuple) else 'none').fillna('none').values.ravel())    
else:
    lee_mpo = LabelEncoder()
    df[to_untuple] = lee_mpo.fit_transform(df[to_untuple].\
        applymap(lambda x: x[0] if isinstance(x, tuple) else 'none').fillna('none').values.ravel())
    save_pickle(lee_mpo, mpo_path)

Saving to: data/chkp5/cache/le_max_prob_object.pkl


#### Histogramy kolorów

In [21]:
# def apply_hist(index):
#     try:
#         frame = cv2.imread("data/images/{}.jpg".format(index))    
#         img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         img = skimage.img_as_ubyte(img)
#         res = {'hist_'+c: exposure.histogram(img[:,:,idx]/255.0, nbins=256)[0] / (img.shape[0]*img.shape[1]) for idx, c in enumerate(['reds', 'greens', 'blues'])}
#         return res
#     except:
#         return {'hist_'+k: np.nan for k in ['reds', 'greens', 'blues']}
    
    
# df = mod_img(df, apply_hist, cache=True, colname='image_histograms')
# df.head(5)

  0%|          | 0/53814 [00:00<?, ?it/s]

Saving to: data/chkp5/cache/image_histograms.pkl


,title,description,channel_title,category_id,tags,views,likes,dislikes,comment_count,thumbnail_link,...,shouting_ratio_description,shouting_ratio_channel_title,channel_title_hash,people_amount,objects,max_prob_object,most_occurence_object,hist_reds,hist_greens,hist_blues
video_id,,,,,,,,,,,,,,,,,,,,,
Y8Ceu9YBR1I,GRACE N MICHELLE REUNITE AFTER 6 YEARS!!!! // ...,Michelle and I reunited AT LAST! And we did a ...,Grace Helbig,22,"[grace, helbig, grace helbig, gracehelbig, gra...",134287.0,9284.0,27.0,910.0,https://i.ytimg.com/vi/Y8Ceu9YBR1I/hqdefault.jpg,...,0.143578,0.181818,8.459502e+11,4.0,"[person, person, person, person]",48,47,"[0.24558449074074074, 0.0314525462962963, 0.01...","[0.27406828703703706, 0.03357060185185185, 0.0...","[0.2353009259259259, 0.015289351851851853, 0.0..."
St6aYO0Gz5U,Matthew Whitaker testifies before House judici...,US acting attorney general Matthew Whitaker te...,Guardian News,25,"[matthew whitaker, whitaker, whitaker testimon...",31382.0,149.0,30.0,18.0,https://i.ytimg.com/vi/St6aYO0Gz5U/hqdefault.jpg,...,0.064125,0.166667,2.520666e+11,8.0,"[tie, person, person, person, person, person, ...",48,47,"[0.09430555555555556, 0.01068287037037037, 0.0...","[0.09864583333333334, 0.010162037037037037, 0....","[0.11651041666666667, 0.017864583333333333, 0...."
k8JuFit-j38,Making Music But Everything Is A Challenge,"I take a plugin which generates ""random beat m...",Levi Niha,10,"[Beat Making, Making A Beat, Music, Music Maki...",230180.0,6358.0,122.0,469.0,https://i.ytimg.com/vi/k8JuFit-j38/hqdefault.jpg,...,0.178049,0.250000,-5.254257e+10,0.0,[],44,78,"[0.23684606481481482, 0.009982638888888888, 0....","[0.24790509259259258, 0.009265046296296296, 0....","[0.24179398148148148, 0.006394675925925926, 0...."
_CzomU7kgFE,Doctor Fate Character Details & Powers Reveale...,Welcome to episode #3 of the DC Universe Annot...,HN Entertainment,24,"[Hybrid Network, HN Entertainment, HNE, Doctor...",1925.0,83.0,4.0,12.0,https://i.ytimg.com/vi/_CzomU7kgFE/hqdefault.jpg,...,0.088915,0.200000,1.813211e+11,1.0,"[dog, person]",48,27,"[0.2644328703703704, 0.02208912037037037, 0.00...","[0.25609953703703703, 0.022002314814814815, 0....","[0.2673668981481481, 0.016655092592592593, 0.0..."
s1bKD0kqPzk,Stephen Fry & Bill Bailey Hilariously Read The...,A little throwback to Stephen Fry & Bill Baile...,The Graham Norton Show,24,"[Graham Norton, Graham Norton Show, The Graham...",756989.0,8855.0,164.0,344.0,https://i.ytimg.com/vi/s1bKD0kqPzk/hqdefault.jpg,...,0.103627,0.210526,7.288290e+11,2.0,"[person, person]",48,47,"[0.22884837962962962, 0.012233796296296296, 0....","[0.3511226851851852, 0.04990740740740741, 0.02...","[0.3956134259259259, 0.05303240740740741, 0.04..."


#### Statystyki kolorów

In [22]:
def apply_image_statistics(index):
    try:
        frame = cv2.imread("data/images/{}.jpg".format(index))    
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = skimage.img_as_ubyte(image)
        
        color_vectors = image.reshape(-1, 3).T
        output = {}
        for c, n in zip(color_vectors, ['red', 'green', 'blue']):
            output['mean_'+n] = np.mean(c)
            output['median_'+n] = np.median(c)
            output['iqr_'+n] = iqr(c)
        return output
    except Exception as ex:
        return {f'{k}_{v}': np.nan for k, v in product(['mean','median','iqr'], ['reds', 'greens', 'blues'])}

df = mod_img(df, apply_image_statistics, cache=True, colname='image_statistics')
df.head(5)

  0%|          | 0/53814 [00:00<?, ?it/s]

Saving to: data/chkp5/cache/image_statistics.pkl


,title,description,channel_title,category_id,tags,views,likes,dislikes,comment_count,thumbnail_link,...,iqr_blue,mean__reds,mean__greens,mean__blues,median__reds,median__greens,median__blues,iqr__reds,iqr__greens,iqr__blues
video_id,,,,,,,,,,,,,,,,,,,,,
Y8Ceu9YBR1I,GRACE N MICHELLE REUNITE AFTER 6 YEARS!!!! // ...,Michelle and I reunited AT LAST! And we did a ...,Grace Helbig,22,"[grace, helbig, grace helbig, gracehelbig, gra...",134287.0,9284.0,27.0,910.0,https://i.ytimg.com/vi/Y8Ceu9YBR1I/hqdefault.jpg,...,201.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
St6aYO0Gz5U,Matthew Whitaker testifies before House judici...,US acting attorney general Matthew Whitaker te...,Guardian News,25,"[matthew whitaker, whitaker, whitaker testimon...",31382.0,149.0,30.0,18.0,https://i.ytimg.com/vi/St6aYO0Gz5U/hqdefault.jpg,...,39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
k8JuFit-j38,Making Music But Everything Is A Challenge,"I take a plugin which generates ""random beat m...",Levi Niha,10,"[Beat Making, Making A Beat, Music, Music Maki...",230180.0,6358.0,122.0,469.0,https://i.ytimg.com/vi/k8JuFit-j38/hqdefault.jpg,...,206.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
_CzomU7kgFE,Doctor Fate Character Details & Powers Reveale...,Welcome to episode #3 of the DC Universe Annot...,HN Entertainment,24,"[Hybrid Network, HN Entertainment, HNE, Doctor...",1925.0,83.0,4.0,12.0,https://i.ytimg.com/vi/_CzomU7kgFE/hqdefault.jpg,...,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s1bKD0kqPzk,Stephen Fry & Bill Bailey Hilariously Read The...,A little throwback to Stephen Fry & Bill Baile...,The Graham Norton Show,24,"[Graham Norton, Graham Norton Show, The Graham...",756989.0,8855.0,164.0,344.0,https://i.ytimg.com/vi/s1bKD0kqPzk/hqdefault.jpg,...,88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Wyrkywanie twarzy i emocji

In [23]:
def process_face(img):
    face_img = transform.resize(img, (48,48,3))
    face_img = color.rgb2gray(face_img)[..., np.newaxis]
    return skimage.img_as_ubyte(face_img)


def apply_face_detection(index):
    try:
        frame = cv2.imread("data/images/{}.jpg".format(index))    
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        faces = face_recognition.face_locations(img)
        detected_faces = [process_face(img[top:bottom, left:right]) for top, right, bottom, left in faces]
        
        if len(faces) == 0:
            result = {'count_'+k.lower(): 0 for k in emotions_to_idx.keys()}
            result['face_count'] = 0
            return result
        else:
            predictions = model.predict(np.stack(detected_faces))
            predictions = [idx_to_emotions[x] for x in np.argmax(predictions, axis=1)]
            
            c = Counter(predictions)
            result = {'count_'+k.lower(): c.get(k, 0) for k in emotions_to_idx.keys()}
            result['face_count'] = len(faces)
            return result
        
    except Exception as ex:
        result = {'count_'+k.lower(): 0 for k in emotions_to_idx.keys()}
        result['face_count'] = 0
        return result
    
df = mod_img(df, apply_face_detection, cache=True, colname='face_detection')
df.head(5)

  0%|          | 0/53814 [00:00<?, ?it/s]

OpenCV(4.5.1) /tmp/pip-req-build-ms668fyv/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

OpenCV(4.5.1) /tmp/pip-req-build-ms668fyv/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

OpenCV(4.5.1) /tmp/pip-req-build-ms668fyv/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

OpenCV(4.5.1) /tmp/pip-req-build-ms668fyv/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

OpenCV(4.5.1) /tmp/pip-req-build-ms668fyv/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

OpenCV(4.5.1) /tmp/pip-req-build-ms668fyv/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

OpenCV(4.5.1) /tmp/pip-req-build-ms668fyv/opencv/modules/imgproc/src/color.cpp:182

,title,description,channel_title,category_id,tags,views,likes,dislikes,comment_count,thumbnail_link,...,iqr__greens,iqr__blues,count_angry,count_sad,count_neutral,count_disgust,count_surprise,count_fear,count_happy,face_count
video_id,,,,,,,,,,,,,,,,,,,,,
Y8Ceu9YBR1I,GRACE N MICHELLE REUNITE AFTER 6 YEARS!!!! // ...,Michelle and I reunited AT LAST! And we did a ...,Grace Helbig,22,"[grace, helbig, grace helbig, gracehelbig, gra...",134287.0,9284.0,27.0,910.0,https://i.ytimg.com/vi/Y8Ceu9YBR1I/hqdefault.jpg,...,NaN,NaN,0,0,0,0,0,2,2,4
St6aYO0Gz5U,Matthew Whitaker testifies before House judici...,US acting attorney general Matthew Whitaker te...,Guardian News,25,"[matthew whitaker, whitaker, whitaker testimon...",31382.0,149.0,30.0,18.0,https://i.ytimg.com/vi/St6aYO0Gz5U/hqdefault.jpg,...,NaN,NaN,1,0,0,0,0,1,0,2
k8JuFit-j38,Making Music But Everything Is A Challenge,"I take a plugin which generates ""random beat m...",Levi Niha,10,"[Beat Making, Making A Beat, Music, Music Maki...",230180.0,6358.0,122.0,469.0,https://i.ytimg.com/vi/k8JuFit-j38/hqdefault.jpg,...,NaN,NaN,0,0,0,0,0,0,0,0
_CzomU7kgFE,Doctor Fate Character Details & Powers Reveale...,Welcome to episode #3 of the DC Universe Annot...,HN Entertainment,24,"[Hybrid Network, HN Entertainment, HNE, Doctor...",1925.0,83.0,4.0,12.0,https://i.ytimg.com/vi/_CzomU7kgFE/hqdefault.jpg,...,NaN,NaN,1,0,0,0,0,0,0,1
s1bKD0kqPzk,Stephen Fry & Bill Bailey Hilariously Read The...,A little throwback to Stephen Fry & Bill Baile...,The Graham Norton Show,24,"[Graham Norton, Graham Norton Show, The Graham...",756989.0,8855.0,164.0,344.0,https://i.ytimg.com/vi/s1bKD0kqPzk/hqdefault.jpg,...,NaN,NaN,0,0,0,0,0,2,0,2


#### Wykrywanie tekstu

In [24]:
# def chunks(lst, n):
#     """Yield successive n-sized chunks from lst."""
#     for i in range(0, len(lst), n):
#         yield lst[i:i + n]
        

# def read_images(ids):
#     return [cv2.cvtColor(cv2.imread("data/images/{}.jpg".format(index)), cv2.COLOR_BGR2RGB) for index in ids]


# if (cache_dir / 'image_text.pkl').exists():
#     kears_ocr_text = load_pickle(kears_ocr_text_path)
# else:
#     kears_ocr_text = []
#     for batch in tqdm(list(chunks(df[~df.thumbnail_link.isna()].index, 24))):
#         prediction_groups = zip(batch, pipeline.recognize(read_images(batch)))
#         kears_ocr_text += [(idx, [word for word, _ in words if len(word) > 3]) for idx, words in prediction_groups]

#     save_pickle(kears_ocr_text, cache_dir / 'image_text.pkl')
    

# df = df.join(pd.Series([v for _, v in kears_ocr_text], index=[k for k, _ in kears_ocr_text], name='image_text'))
# df.head()

In [25]:
# def apply_text_detection(index):
#     try:
#         frame = cv2.imread("data/images/{}.jpg".format(index))    
#         img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         words = pipeline.recognize([img])[0]
        
#         kears_ocr_text = [word for word, _ in words if len(word) > 3]
#         return {'image_text': kears_ocr_text}
                
#     except Exception as ex:
#         print(ex)
#         return {'image_text': []}
    
# df = mod_img(df, apply_text_detection, cache=True, colname='image_text')
# df.head(5)

### Embeddingi

#### Opisy i tytuły

In [26]:
# #  Desciption and title embeddings
# model_description = gensim.models.Word2Vec.load('models/model_description.model')
# model_title = gensim.models.Word2Vec.load('models/model_title.model')

# stopwords_list = list(stopwords.words('english'))   

# def clear_sentence(sentence):
#     try:
#         sentence = re.sub(r"http\S+", "", sentence)
#         sentence = word_tokenize(re.sub('[^A-Za-z]+', ' ', sentence.lower()))
#         sentence_temp = []
#         for word in sentence:
#             if (word not in stopwords_list) and (len(word) > 2) and not (word.isnumeric()):
#                 sentence_temp.append(word)
#         return sentence_temp
#     except TypeError:
#         return []

# descriptions = df.apply(lambda w: clear_sentence(w['description']), axis=1).values
# titles = df.apply(lambda w: clear_sentence(w['title']), axis=1).values

# def word_averaging(wv, sequence):
#     mean = []
#     for word in sequence:
#         if word in wv.vocab: mean.append(wv.get_vector(word))
#     if not mean:
#         return np.zeros(200,)
#     return gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)

# def word_averaging_list(wv, sequences):
#     return np.vstack([word_averaging(wv, w) for w in sequences])

# descriptions = word_averaging_list(model_description.wv, descriptions)
# titles = word_averaging_list(model_title.wv, titles)

# df['description_embedding'] = list(descriptions)
# df['title_embedding'] = list(titles)
# df.head(3)

#### Tagi i obiekty

In [27]:
# df_temp = deepcopy(df[text_to_process])

# def word_averaging(wv, sequence):
#     mean = []
#     for word in sequence:
#         if word in wv.vocab: mean.append(wv.get_vector(word))
#     if not mean:
#         return np.zeros(300,)
#     return gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)

# def word_averaging_list(wv, sequences):
#     return np.vstack([word_averaging(wv, w) for w in sequences])

# def embedding_model(text, name):
#     # Word averaging
#     wa = word_averaging_list(wv, text)
#     # Replace cols
#     df_temp[name] = list(wa)

# for col in text_to_process:
#     embedding_model(df_temp[col].values, col)

# df[text_to_process] = df_temp
# df[text_to_process].head(3)

### Sprawdzenie listy Top 50 tagów

In [28]:
all_tags = [tag.lower() for tag in df.tags.apply(pd.Series).stack().values if tag.lower() != '[none]']
top_tags = [k for k, _ in Counter(all_tags).most_common(50)]

for top_tag in top_tags:
    df[f'has_{top_tag}'] = df.tags.apply(lambda x: isinstance(x, list) and top_tag in x).astype('int8')

df['has_no_tags'] = df.tags.apply(lambda x: x == '[none]' or (isinstance(x, list) and '[none]' in x)).astype('int8')
df.head(5)

,title,description,channel_title,category_id,tags,views,likes,dislikes,comment_count,thumbnail_link,...,has_cooking,has_football,has_pop,has_film,has_new,has_funny videos,has_fashion,has_recipe,has_fun,has_no_tags
video_id,,,,,,,,,,,,,,,,,,,,,
Y8Ceu9YBR1I,GRACE N MICHELLE REUNITE AFTER 6 YEARS!!!! // ...,Michelle and I reunited AT LAST! And we did a ...,Grace Helbig,22,"[grace, helbig, grace helbig, gracehelbig, gra...",134287.0,9284.0,27.0,910.0,https://i.ytimg.com/vi/Y8Ceu9YBR1I/hqdefault.jpg,...,0,0,0,0,0,0,0,0,0,0
St6aYO0Gz5U,Matthew Whitaker testifies before House judici...,US acting attorney general Matthew Whitaker te...,Guardian News,25,"[matthew whitaker, whitaker, whitaker testimon...",31382.0,149.0,30.0,18.0,https://i.ytimg.com/vi/St6aYO0Gz5U/hqdefault.jpg,...,0,0,0,0,0,0,0,0,0,0
k8JuFit-j38,Making Music But Everything Is A Challenge,"I take a plugin which generates ""random beat m...",Levi Niha,10,"[Beat Making, Making A Beat, Music, Music Maki...",230180.0,6358.0,122.0,469.0,https://i.ytimg.com/vi/k8JuFit-j38/hqdefault.jpg,...,0,0,0,0,0,0,0,0,0,0
_CzomU7kgFE,Doctor Fate Character Details & Powers Reveale...,Welcome to episode #3 of the DC Universe Annot...,HN Entertainment,24,"[Hybrid Network, HN Entertainment, HNE, Doctor...",1925.0,83.0,4.0,12.0,https://i.ytimg.com/vi/_CzomU7kgFE/hqdefault.jpg,...,0,0,0,0,0,0,0,0,0,0
s1bKD0kqPzk,Stephen Fry & Bill Bailey Hilariously Read The...,A little throwback to Stephen Fry & Bill Baile...,The Graham Norton Show,24,"[Graham Norton, Graham Norton Show, The Graham...",756989.0,8855.0,164.0,344.0,https://i.ytimg.com/vi/s1bKD0kqPzk/hqdefault.jpg,...,0,0,0,0,0,0,0,0,0,0


In [39]:
df.to_pickle('data/chkp5.pkl')

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53814 entries, Y8Ceu9YBR1I to 4YFo4bdMO8Q
Columns: 121 entries, title to has_no_tags
dtypes: float32(1), float64(26), int64(34), int8(51), object(9)
memory usage: 33.6+ MB


### Kolumny NaN

In [41]:
df.isna().sum()[df.isna().sum() > 0]

description                    144
views                            9
likes                          325
dislikes                       325
comment_count                 1253
description_word_count         144
description_url_count          144
shouting_ratio_description     144
people_amount                  825
objects                        825
hist_reds                      825
hist_greens                    825
hist_blues                     825
mean_red                       825
median_red                     825
iqr_red                        825
mean_green                     825
median_green                   825
iqr_green                      825
mean_blue                      825
median_blue                    825
iqr_blue                       825
dtype: int64